 # 2021年9月23日 该模型用于问题300的数据生成
该WCGAN网络模型处理的数据是一周的用电数据，
生成一周的用电数据 即D网络的输入数据是7*24+2位标签，输出是1或0，判断该模型是否为真实数据 G网络的输入是（1 100 ）的随机噪声和2位的随机标签，输出是724的用电数据

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
from torchvision.transforms import transforms

In [2]:
torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
LEARNING_RATE = 1e-4
BATCH_SIZE=32
IMG_SIZE_L=7
IMG_SIZE_H=48
CHANNELS_IMG=1
NUM_CLASSES = 4
GEN_EMBEDDING = 100
Z_DIM=100
NUM_EPOCHS=30
FEATURES_DISC=16
FEATURES_GEN=16
CRITIC_ITERATIONS =5
LAMBDA_GP = 5

In [4]:
class MyDataSet(Dataset):
    def __init__(self,csv_file):
        self.data_df=pd.read_csv(csv_file,header=0)
        #self.transform = transofrms
        pass
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self,index):
        label = int(self.data_df.iloc[index,1])
        target = torch.zeros((4))
        target[label]=1.0
        
        image_values = torch.FloatTensor(self.data_df.iloc[index,10:].values)/12.469
        image_values = torch.FloatTensor(image_values).view(1,7,48)
        sample = {'label':label,'image_values':image_values,'target':target}
        #return sample
        return label,image_values,target,sample
        
    
    def plot_image(self,index):
        arr = torch.FloatTensor(self.data_df.iloc[index,10:].values).reshape(7,48)/12.469
        plt.title("label=" + str(self.data_df.iloc[index,1]))
        
        plt.imshow(arr,interpolation = 'none',cmap = 'Blues')
        #plt.savefig('WeeklyData7*48.pdf',bbox_inches='tight')
        pass
    
#     def plot_figure(self,index):
#         y1 = torch.FloatTensor(self.data_df.iloc[index,10:].values)
#         x1 = range(1,337)
#         plt.title("label=" + str(self.data_df.iloc[index,2]))
#         plt.plot(x1,y1)
#         #plt.savefig('WeeklyData336.pdf',bbox_inches='tight')
#          plt.show()
        
#         pass
    pass

In [5]:
torch_dataset= MyDataSet('./Consumption_data_to_train_WDCGAN*.csv')

In [6]:
loader =Data.DataLoader(torch_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [7]:
def gradient_penalty(critic,labels,real,fake,device="cpu"):
    BATCH_SIZE,C,H,W = real.shape
    epsilon = torch.rand((BATCH_SIZE,1,1,1)).repeat(1,C,H,W).to(device)
    interpolated_images = real*epsilon + fake*(1-epsilon)
    
    mixed_scores = critic(interpolated_images,labels)
    
    gradient = torch.autograd.grad(
        inputs = interpolated_images,
        outputs = mixed_scores,
        grad_outputs = torch.ones_like(mixed_scores),
        create_graph = True,
        retain_graph = True,      
    )[0]
    
    
    gradient = gradient.view(gradient.shape[0],-1)
    gradient_norm = gradient.norm(2,dim =1)
    gradient_penalty = torch.mean((gradient_norm -1)**2)
    return gradient_penalty

In [8]:
class Discriminator(nn.Module):
    def __init__(self,channels_img,features_d,num_classes,img_size_L,img_size_H):
        super(Discriminator,self).__init__()
        self.img_size_L = img_size_L
        self.img_size_H = img_size_H
        self.disc = nn.Sequential(
                              # input N*channels_img*7*48
            nn.Conv2d(channels_img+1,features_d,kernel_size=2,stride=2,padding=1), #4*25
            nn.LeakyReLU(0.2),
            
            self._block(features_d,features_d*2,2,2,0),#2*12
            self._block(features_d*2,features_d*4,2,2,0),#1*6
            nn.Conv2d(features_d*4,1,kernel_size=(1,6),stride=1,padding=0), #1*1       
                
        )
        self.embed = nn.Embedding(num_classes,img_size_L*img_size_H)
    def _block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
        nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        bias=False,),
        nn.InstanceNorm2d(out_channels,affine=True),
        nn.LeakyReLU(0.2),
        )
    def forward(self,x,labels):
        
        embedding = self.embed(labels).view(labels.shape[0],1,self.img_size_L,self.img_size_H)
        x = torch.cat([x,embedding],dim=1)
        return self.disc(x)

In [9]:
class Generator(nn.Module):
    def __init__(self,
                 channels_noise,
                 channels_img,
                 features_g,
                 num_classes,
                 img_size_L,
                 img_size_H,                 
                 embed_size,
                ):
        super(Generator,self).__init__()
        self.img_size_L = img_size_L
        self.img_size_H = img_size_H
        self.gen = nn.Sequential(
                            # input N*200*1*1
            self._block(channels_noise+embed_size,features_g*16,(1,6),1,0),#N*256*1*6
            self._block(features_g*16,features_g*8,2,2,0),#N*128*2*12
            self._block(features_g*8,features_g*4,2,2,0),#N*64*4*24
            
            
            nn.ConvTranspose2d(features_g*4,channels_img,kernel_size=(3,2),stride=2,padding=(1,0)) ,#N*1*7*48     
            nn.Tanh(),#[-1,1]
        )
        self.embed = nn.Embedding(num_classes,embed_size)
        
    def _block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
        nn.ConvTranspose2d(
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        )
    def forward(self,x,labels):
        #latent vector z:N*noise_dim*1*1
        embedding = self.embed(labels).unsqueeze(2).unsqueeze(3)
        x = torch.cat([x,embedding],dim =1)
        return self.gen(x)

In [10]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m,(nn.Conv2d,nn.ConvTranspose2d,nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data,0.0,0.02)

In [11]:
gen = Generator(Z_DIM,CHANNELS_IMG,FEATURES_GEN,NUM_CLASSES,IMG_SIZE_L,IMG_SIZE_H,GEN_EMBEDDING).to(device)
critic = Discriminator(CHANNELS_IMG,FEATURES_GEN,NUM_CLASSES,IMG_SIZE_L,IMG_SIZE_H).to(device)
initialize_weights(gen)
initialize_weights(critic)

In [12]:
opt_gen = optim.Adam(gen.parameters(),lr=LEARNING_RATE,betas=(0.0,0.999))
opt_critic = optim.Adam(critic.parameters(),lr=LEARNING_RATE,betas=(0.0,0.999))

In [13]:
fixed_noise = torch.randn(32,Z_DIM,1,1).to(device)
writer_real = SummaryWriter(f"WGANs/displcy_the_real")
writer_fake = SummaryWriter(f"WGANs/displcy_the_fake")
step = 0

In [14]:
gen.train()
critic.train()

for epoch in range(NUM_EPOCHS):
     
    for batch_idx,data in enumerate(loader):
        real = data[1]
        real = real.to(device)
        cur_batch_size = real.shape[0]
        labels = data[0]
        labels = labels.to(device)
#     for batch_idx, (real,labels) in enumerate(loader):
#         real = real.to(device)
#         cur_batch_size = real.shape[0]
#         labels = labels.to(device)
        
        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn((cur_batch_size,Z_DIM,1,1)).to(device)
            fake = gen(noise,labels)
            critic_real = critic(real,labels).reshape(-1)
            critic_fake = critic(fake,labels).reshape(-1)
            gp = gradient_penalty(critic,labels,real,fake,device=device)
            loss_critic = ( -(torch.mean(critic_real)-torch.mean(critic_fake)) +LAMBDA_GP*gp)
            critic.zero_grad()
            loss_critic.backward(retain_graph = True)
            opt_critic.step()
     
        
        ###训练生成器 min log(1-D(G(z))) <----->max log(D(G(z))) 
        gen_fake = critic(fake,labels).reshape(-1)
        loss_gen = -torch.mean(gen_fake)        
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        
        # Print losses osccasionally and print to tensorboard
        if batch_idx %100==0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)}\
                Loss D: {loss_critic:.4f},loss G: {loss_gen:.4f}"
            )
            
            with torch.no_grad():
                fake = gen(noise,labels)
                #take out(up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32],normalize = True
                
                )
                
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32],normalize = True
                
                )
                
                writer_real.add_image("Real",img_grid_real,global_step=step)
                writer_fake.add_image("Fake",img_grid_fake,global_step=step)
                
            step +=1

Epoch [0/30] Batch 0/1250                Loss D: 0.3775,loss G: 0.0041
Epoch [0/30] Batch 100/1250                Loss D: -0.1152,loss G: 0.9593
Epoch [0/30] Batch 200/1250                Loss D: -0.1199,loss G: 0.6394
Epoch [0/30] Batch 300/1250                Loss D: -0.0656,loss G: 0.6196
Epoch [0/30] Batch 400/1250                Loss D: -0.0773,loss G: 0.0513
Epoch [0/30] Batch 500/1250                Loss D: -0.0805,loss G: -0.0647
Epoch [0/30] Batch 600/1250                Loss D: -0.0480,loss G: -0.2418
Epoch [0/30] Batch 700/1250                Loss D: -0.0752,loss G: -0.6688
Epoch [0/30] Batch 800/1250                Loss D: -0.0635,loss G: -0.8514
Epoch [0/30] Batch 900/1250                Loss D: -0.0697,loss G: -0.9381
Epoch [0/30] Batch 1000/1250                Loss D: -0.0577,loss G: -0.6677
Epoch [0/30] Batch 1100/1250                Loss D: -0.0573,loss G: -0.9241
Epoch [0/30] Batch 1200/1250                Loss D: -0.0179,loss G: -0.8481
Epoch [1/30] Batch 0/1250     

Epoch [8/30] Batch 600/1250                Loss D: -0.0218,loss G: -5.6562
Epoch [8/30] Batch 700/1250                Loss D: -0.0117,loss G: -5.9799
Epoch [8/30] Batch 800/1250                Loss D: -0.0469,loss G: -5.0836
Epoch [8/30] Batch 900/1250                Loss D: 0.0032,loss G: -5.6267
Epoch [8/30] Batch 1000/1250                Loss D: 0.0047,loss G: -6.0664
Epoch [8/30] Batch 1100/1250                Loss D: -0.0371,loss G: -5.5047
Epoch [8/30] Batch 1200/1250                Loss D: -0.0052,loss G: -5.7989
Epoch [9/30] Batch 0/1250                Loss D: -0.0644,loss G: -5.4274
Epoch [9/30] Batch 100/1250                Loss D: 0.0103,loss G: -5.6897
Epoch [9/30] Batch 200/1250                Loss D: -0.0543,loss G: -6.6217
Epoch [9/30] Batch 300/1250                Loss D: -0.0388,loss G: -6.3422
Epoch [9/30] Batch 400/1250                Loss D: -0.0272,loss G: -6.3342
Epoch [9/30] Batch 500/1250                Loss D: -0.0259,loss G: -6.6392
Epoch [9/30] Batch 600/1250

Epoch [16/30] Batch 1100/1250                Loss D: -0.0305,loss G: -10.2047
Epoch [16/30] Batch 1200/1250                Loss D: -0.0905,loss G: -9.2802
Epoch [17/30] Batch 0/1250                Loss D: -0.0224,loss G: -10.2763
Epoch [17/30] Batch 100/1250                Loss D: -0.0405,loss G: -10.0164
Epoch [17/30] Batch 200/1250                Loss D: -0.0173,loss G: -9.5704
Epoch [17/30] Batch 300/1250                Loss D: -0.0125,loss G: -10.5101
Epoch [17/30] Batch 400/1250                Loss D: -0.0242,loss G: -10.2340
Epoch [17/30] Batch 500/1250                Loss D: -0.0180,loss G: -11.4145
Epoch [17/30] Batch 600/1250                Loss D: 0.0149,loss G: -12.4856
Epoch [17/30] Batch 700/1250                Loss D: -0.0306,loss G: -11.1520
Epoch [17/30] Batch 800/1250                Loss D: -0.0052,loss G: -11.3938
Epoch [17/30] Batch 900/1250                Loss D: -0.0324,loss G: -11.6689
Epoch [17/30] Batch 1000/1250                Loss D: -0.0451,loss G: -10.9389
E

Epoch [25/30] Batch 100/1250                Loss D: 0.0519,loss G: -16.5800
Epoch [25/30] Batch 200/1250                Loss D: -0.0065,loss G: -15.7906
Epoch [25/30] Batch 300/1250                Loss D: -0.0636,loss G: -15.1901
Epoch [25/30] Batch 400/1250                Loss D: -0.0523,loss G: -15.7195
Epoch [25/30] Batch 500/1250                Loss D: -0.0051,loss G: -15.4518
Epoch [25/30] Batch 600/1250                Loss D: -0.0184,loss G: -16.5264
Epoch [25/30] Batch 700/1250                Loss D: -0.0310,loss G: -15.3931
Epoch [25/30] Batch 800/1250                Loss D: 0.0114,loss G: -14.3057
Epoch [25/30] Batch 900/1250                Loss D: 0.0002,loss G: -13.9564
Epoch [25/30] Batch 1000/1250                Loss D: -0.0489,loss G: -15.7711
Epoch [25/30] Batch 1100/1250                Loss D: -0.0245,loss G: -14.8887
Epoch [25/30] Batch 1200/1250                Loss D: -0.0130,loss G: -14.4531
Epoch [26/30] Batch 0/1250                Loss D: 0.0131,loss G: -15.0216
Ep

In [15]:
torch.save(gen.state_dict(),'./output/WCGAN_gen.pth')
torch.save(critic.state_dict(),'./output/WCGANcritic.pth')

In [14]:
chechpoint=torch.load('./output/WCGAN_gen.pth')
gen = Generator(Z_DIM,CHANNELS_IMG,FEATURES_GEN,NUM_CLASSES,IMG_SIZE_L,IMG_SIZE_H,GEN_EMBEDDING).to(device)
gen.load_state_dict(chechpoint)

<All keys matched successfully>

In [15]:
####Used for generating sythetic data
####For the CER Dataset,Question 300 has 3 different labels,we set the input label of generator to control the output class
#### Here we generate the data with label 1
batch_out_data = 30000
out_noise_1 = torch.randn((batch_out_data,100,1,1))
out_1 = torch.ones(batch_out_data)
out_1 = torch.FloatTensor(out_1).long()
out_noise_1=out_noise_1.to(device)
out_1 = out_1.to(device)
fake1 = gen(out_noise_1,out_1).to(device)
print(fake1.shape)
L_1 = fake1.view(batch_out_data,7*48)*12.469
L_1=L_1.cpu().detach().numpy().tolist()
np.savetxt('./Generated_Data_30000/Q300_1.csv',L_1,delimiter=',')


torch.Size([30000, 1, 7, 48])


In [15]:
####Used for generating sythetic data
####For the CER Dataset,Question 300 has 3 different labels,we set the input label of generator to control the output class
#### Here we generate the data with label 2
batch_out_data = 30000
out_noise_2 = torch.randn((batch_out_data,100,1,1))
out_2 = torch.ones(batch_out_data)*2
out_2 = torch.FloatTensor(out_2).long()
out_noise_2=out_noise_2.to(device)
out_2 = out_2.to(device)
fake2 = gen(out_noise_2,out_2).to(device)
print(fake2.shape)
L_2 = fake2.view(batch_out_data,7*48)*12.469
L_2=L_2.cpu().detach().numpy().tolist()
np.savetxt('./Generated_Data_30000/Q300_2.csv',L_2,delimiter=',')

torch.Size([30000, 1, 7, 48])


In [15]:
####Used for generating sythetic data
####For the CER Dataset,Question 300 has 3 different labels,we set the input label of generator to control the output class
#### Here we generate the data with label 3
batch_out_data = 30000
out_noise_3 = torch.randn((batch_out_data,100,1,1))
out_3 = torch.ones(batch_out_data)*3
out_3 = torch.FloatTensor(out_3).long()
out_noise_3=out_noise_3.to(device)
out_3 = out_3.to(device)
fake3 = gen(out_noise_3,out_3).to(device)
print(fake3.shape)
L_3 = fake3.view(batch_out_data,7*48)*12.469
L_3=L_3.cpu().detach().numpy().tolist()
np.savetxt('./Generated_Data_30000/Q300_3.csv',L_3,delimiter=',')

torch.Size([30000, 1, 7, 48])
